In [5]:
# Audio Concatenator
#
# This program is for joining together multiple audio files in a folder. It will take in a folder name, check the folder for all the .mp3 files
# and then concatenate them all together to make one big .mp3 file. This is mainly being used for audiobooks and concatenating the individual
# files together into a single HUGE file.
#
# Special Notes: 
#   - You must have ffmpeg, ffprobe, and ffplay installed on your computer in order for this to work!
#       - https://www.youtube.com/watch?v=H1o6MWnmwpY
#
# Bugs:
#   - Only can convert less than 400MB of files? or some amount like that
#       - Might have to call ffmpeg directly using system os! https://www.reddit.com/r/pythonhelp/comments/upu8y8/trouble_with_combining_large_audio_files_using/
#       - Make a .txt file with a list of all the different files you make in the temp folder and then concatenate them together
#
# TODO:
#   - Maybe to complete this i need to combine 1/4 of the files at a time at the 16000 quality in a different directory, then combine those files at full quality
#     and export them to the original folder and delete the temp folder.
#   - Make it be able to accept WAV files and see if it can convert larger file sizes for WAV
#   - Make into a function / GUI that only takes in folderPath, exportFileName, and fileType
#   - Upload to GitHub
#   - Multithreading
#   - ffmpeg might be WAY faster than pydub...
#
# @author  Scott Merkley
# @version 20230511

import os
from os import listdir
import subprocess
import time
import eyed3
from eyed3.id3.frames import ImageFrame
from pydub import AudioSegment
from pydub.playback import play
import numpy as np


# User input
folderPath = "/Users/smerkley95/Desktop/The Innovators - Walter Isaacson"
exportFileName = "The Innovators by Walter Isaacson"
exportFormat = "MP3"
coverImage = "cover.jpg"



allFileNames = sorted(listdir(folderPath))

print("Finding all file names with .mp3...")
allFileNamesWithMP3 = []
for fileName in allFileNames:
    if fileName[-4:] == ".mp3":
        allFileNamesWithMP3.append(fileName)

print("Exporting audio...")
# Making a list of files for ffmpeg to concatenate
exportListName = folderPath + "/" + exportFileName + "_LIST.txt"
f = open(exportListName, "w")
for fileName in allFileNamesWithMP3:
    f.write("file \'" + folderPath + "/" + fileName + "\'" + "\n")
f.close()

# Make a call for ffmpeg and run it using shell
fileExportName = folderPath + "/" + exportFileName + ".mp3"
cmd_str = "ffmpeg -f concat -safe 0 -i " + exportListName.replace(" ", "\ ") + " -c copy " + fileExportName.replace(" ", "\ ")
p = subprocess.run(cmd_str, shell = True)
os.remove(exportListName)


print("Adding cover artwork to audio file...")
audioFile = eyed3.load(folderPath + '/' + exportFileName + ".mp3")
if audioFile.tag == None:
    audioFile.initTag()
audioFile.tag.images.set(ImageFrame.FRONT_COVER, open(folderPath + '/' + coverImage, 'rb').read(), 'image/jpeg')
audioFile.tag.save()

print("Done.")


Finding all file names with .mp3...
Exporting audio...
Adding cover artwork to file...
Done.


In [7]:
# Usefule Code


# fileName = "/Users/smerkley95/Desktop/Napoleon Hill - Think and Grow Rich - Unabridged Version/Napoleon Hill - Think and Grow Rich - Chapter 00 Part 1 (What Do You Want Most).mp3"
# song = AudioSegment.from_mp3(fileName)
# play(song)

In [22]:
# You have to export the file names without spaces by changing spaces from " " to "\ " 👍
print(fileExportName.replace(" ", "\ "))

/Users/smerkley95/Desktop/TEST/The\ Innovators\ by\ Walter\ Isaacson.mp3
